In [1]:
import tweepy

In [3]:
auth = tweepy.auth.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
twitter_api = tweepy.API(auth)

In [4]:
print(twitter_api)

In [5]:
# The Yahoo! Where On Earth ID for the entire world is 1.
# See https://dev.twitter.com/docs/api/1.1/get/trends/place and
# http://developer.yahoo.com/geo/geoplanet/

WORLD_WOE_ID = 1
ZH_WOE_ID = 784794

world_trends = twitter_api.trends_place(WORLD_WOE_ID)
zh_trends = twitter_api.trends_place(ZH_WOE_ID)

#print(zh_trends)

In [6]:
for trend in zh_trends[0]['trends']:
    print(trend['name'])

#grag
#1000ernote
Luke Perry
#IWD2019
#WomensDay
#RIPLukePerry
Fragen
Keith Flint
Berne
Beverly Hills
Riverdale
Saison
Massnahmen
sturm
Studie
Klar
michael jackson
rest
Sprache
stimmen
bienne
J'avais
Vernets
Aussage
runde
#GSHCZSC
#NationalLeague
#theprodigy
#WomensWeek
#prodigy
#bennet
#children
#africa
#playoffs
#Azerbaijan
#SwissGames
#aargau
#GenderEquality
#Digitalisierung
#Artikel13
#Vaud
#childrights
#hcap
#Syria
#Israel
#Macron
#srffussball
#srfhockey
#pakistan
#climatechange
